In [1]:
import pandas as pd
from selenium import webdriver 
from selenium.webdriver.support.ui import WebDriverWait
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.support import expected_conditions as EC 
from selenium.webdriver.common.keys import Keys  
from selenium.webdriver.support.ui import Select 
from selenium.webdriver.common.by import By 
from selenium.common.exceptions import NoSuchElementException  #Este importé para manejar más errores.
from time import sleep
import os
import signal
import sys

In [2]:
df_peliculas = pd.read_csv("peliculas_api.csv")

df_peliculas.head(10)


,Unnamed: 0,Tipo,Nombre,Año de Estreno,Mes de Estreno,ID de la Película,Genero
0,0,Movie,Pál Adrienn,2011,6,tt0146592,Accion/Drama
1,1,Movie,Oda az igazság,2010,2,tt0154039,Accion/Drama
2,2,Movie,A zöld sárkány gyermekei,2011,4,tt0162942,Accion/Drama
3,3,Movie,The Final Journey,2010,9,tt0230212,Accion/Drama
4,4,Movie,In My Sleep,2010,4,tt0326965,Accion/Drama
5,5,Movie,This Wretched Life,2010,7,tt0331312,Accion/Comedia/Drama
6,6,Movie,Blind Sided,Desconocido,Desconocido,tt0337882,Accion/Comedia/Drama
7,7,Movie,Anderson's Cross,2010,5,tt0393049,Accion/Comedia/Drama
8,8,Movie,Den milde smerte,2014,5,tt0396123,Accion/Drama
9,9,Movie,Capture the Flag,Desconocido,Desconocido,tt0402887,Accion/Drama


In [3]:
df_peliculas.columns

Index(['Unnamed: 0', 'Tipo', 'Nombre', 'Año de Estreno', 'Mes de Estreno',
       'ID de la Película', 'Genero'],
      dtype='object')

## Extraccion datos imdb y Rotten Tomatoes:

In [4]:
df_id_nombre = df_peliculas[['ID de la Película', 'Nombre']]
df_id_nombre


,ID de la Película,Nombre
0,tt0146592,Pál Adrienn
1,tt0154039,Oda az igazság
2,tt0162942,A zöld sárkány gyermekei
3,tt0230212,The Final Journey
4,tt0326965,In My Sleep
...,...,...
34970,tt31308988,Married Life
34971,tt31314922,Concept High
34972,tt31316077,Satisfaction
34973,tt31319755,Party Pooper


### Fuera de la función:

In [5]:
# driver = webdriver.Chrome()


# datos_peliculas = {
#     "Nombre": [],
#     "Puntuacion_IMDB": [],
#     "Puntuacion_Rotten_Tomatoes": [],
#     "Director": [],
#     "Guionistas": [],
#     "Argumento": [],
#     "Duracion": []
# }

# # Extracción de datos. en la linea de abajo cambiamos el rango de cada busqueda con slicing. 'df_peliculas[x:y]'
# for indice, fila in df_peliculas.iterrows():   # metodo pandas que itera entre las filas
#     imdb_id = fila['ID de la Película']
#     nombre_pelicula = fila['Nombre']
#     try:
#         driver.get(f"https://www.imdb.com/title/{imdb_id}")
#         sleep(2)
#         driver.maximize_window()
#         driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
#         sleep(1)
#     except NoSuchElementException:
#         # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
#         print("No se requiere aceptación de cookies.")
#     except NoSuchElementException: #Herramienta importada de Selenium.
#          print(f"No se encontro la pagina para el ID {imdb_id}")
#     except Exception as e:
#          print(f"Error durante la búsqueda del ID IMDB {imdb_id}: {str(e)}")
         
        
#     try:
#         puntuacion_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text
#     except Exception as e:
#         print(f"No se encontro la puntuacion de la {nombre_pelicula} en imdb {str(e)}")
#         puntuacion_imdb = "N/A"
        
#     try:
#         director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div").text
#     except Exception as e:
#         print(f"No se encontro el director de {nombre_pelicula} en imdb {str(e)}")
#         director = "N/A"
        
#     try:
#         guionistas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div").text
#     except Exception as e:
#         print(f"No se encontro guionista de {nombre_pelicula} en imdb {str(e)}")
#         guionistas = "N/A"
        
#     try:  # Esto lo hacemos porque hay paginas de imdb que tienen el argumento en diferentes paths.
#         try: 
#             argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-10.dwKwMe > section > p > span.sc-7193fc79-2.kpMXpM").text
#         except NoSuchElementException:
#             try:
#                 argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-10.hERErh > section > p > span.sc-7193fc79-0.ftEVcu").text
#             except NoSuchElementException:
#                 try:
#                     argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.eqrmzR > div.sc-92625f35-6.cApiRN > div.sc-92625f35-10.hERErh > section > p > span.sc-7193fc79-2.kpMXpM").text
#                 except NoSuchElementException:
#                     try:
#                         argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.kiNxNJ > div.sc-92625f35-6.gFpQdY > div.sc-92625f35-10.cDqtIi > section > p > span.sc-7193fc79-2.kpMXpM").text
#                     except NoSuchElementException:
#                         try:
#                             argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-10.hERErh > section > p").text
#                         except NoSuchElementException:
#                             argumento = "N/A"
                              
#     except Exception as e:
#             print(f"No se encontro argumento de {nombre_pelicula} en imdb {str(e)}")
#             argumento = "N/A"
#     try:    
#         try:
#             duracion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)").text
#         except NoSuchElementException: # Esto lo ponemos porque hay paginas de imdb que tienen la direccion en diferentes paths.
#             try:
#                 duracion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2)").text
#             except NoSuchElementException:
#                 duracion = "N/A"
#     except Exception as e:
#             print(f"No se encontro duracion de {nombre_pelicula} en imdb {str(e)}")
#             duracion = "N/A"
        

#     # ROTTEN TOMATOES buscamos por nombre porque no acepta IDS a diferencia de IMD
#     try:
#         nombre_pelicula_modificado = nombre_pelicula.replace("'","").replace("ñ","n").replace(":","").replace(",","").replace("/","").replace("!","").replace("?","") # porque no me deja hacerlo directamente en la siguiente línea.
#         driver.get(f'https://www.rottentomatoes.com/m/{nombre_pelicula_modificado.replace(" ","_").replace("&","and").lower()}')
#         sleep(2)
#         try:
#             driver.find_element("css selector", "#onetrust-accept-btn-handler").click()
#         except NoSuchElementException:
#         # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
#             print("No se requiere aceptación de cookies.")
        
#         puntuacion_rotten = driver.find_element("css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button > rt-text").text
            
#     except Exception as e:
#         print(f"No se encontro la película {nombre_pelicula} en Rotten Tomatoes: {str(e)}")
#         puntuacion_rotten = "N/A"
        
    

#     datos_peliculas['Nombre'].append(nombre_pelicula)
#     datos_peliculas['Puntuacion_IMDB'].append(puntuacion_imdb)
#     datos_peliculas['Puntuacion_Rotten_Tomatoes'].append(puntuacion_rotten)
#     datos_peliculas['Director'].append(director)
#     datos_peliculas['Guionistas'].append(guionistas)
#     datos_peliculas['Argumento'].append(argumento)
#     datos_peliculas['Duracion'].append(duracion)


# driver.close()

# datos_peliculas


## Dentro de una función:
Las dos primeras funciones son para guardar los datos en  archivos .csv en caso de interrupcion o error en el 'scraping'.

In [6]:
def signal_handler(signal, frame, driver, datos_peliculas):
    print("Proceso interrumpido. Guardando datos...")
    df_resultados_temporales = pd.DataFrame(datos_peliculas)
    df_resultados_temporales.to_csv('datos_temporales.csv', index=False)
    print("Datos guardados en 'datos_temporales.csv'.")
    guardar_datos(datos_peliculas)  #  definir esta función
    driver.close()
    sys.exit(0)
    
def guardar_datos(datos_peliculas):
   # Define el  archivo donde vamos a guardar los datos.
    ruta_archivo = 'fase-2-datos-peliculas-imdb-rotten.csv'
    
    # Convierte el diccionario a DataFrame
    df_nuevos_datos = pd.DataFrame(datos_peliculas)
    
    if os.path.exists(ruta_archivo):
        # Si el archivo ya existe, carga los datos existentes y los junta.
        df_existentes = pd.read_csv(ruta_archivo)
        df_resultante = pd.concat([df_existentes, df_nuevos_datos], ignore_index=True)
    else:
        # Si el archivo no existe, utiliza los nuevos datos directamente
        df_resultante = df_nuevos_datos
    
    # Guardamos el DataFrame resultante en el archivo, sobrescribiendo el archivo existente
    df_resultante.to_csv(ruta_archivo, index=False)
    
    print(f"Datos guardados o actualizados exitosamente en {ruta_archivo}.")

# Aqui empieza nuestra función de extraccion de datos 'IMDB' y 'ROTTEN TOMATOES'
def extraccion_imdb_rotten(df_peliculas):
    driver = webdriver.Chrome()
    datos_peliculas = {
        "Nombre": [],
        "Puntuacion_IMDB": [],
        "Puntuacion_Rotten_Tomatoes": [],
        "Director": [],
        "Guionistas": [],
        "Argumento": [],
        "Duracion": []
    }
    
    signal.signal(signal.SIGINT, lambda sig, frame: signal_handler(sig, frame, driver, datos_peliculas))  #Esto es para poder interrumpir la función.

    for indice, fila in df_peliculas[11310:].iterrows(): # Aqui podemas aplicar  el rango de búsqueda usando 'slicing', para no tener que empezar de cero en caso de error o interrupción.
        imdb_id = fila['ID de la Película']
        nombre_pelicula = fila['Nombre']
        try:
            driver.get(f"https://www.imdb.com/title/{imdb_id}")
            sleep(2)
            driver.maximize_window()
            driver.find_element("css selector", "#__next > div > div > div.sc-jrcTuL.bPmWiM > div > button.icb-btn.sc-bcXHqe.sc-dkrFOg.sc-iBYQkv.dcvrLS.ddtuHe.dRCGjd").click()
            sleep(1)
        except NoSuchElementException:
            # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
            print("No se requiere aceptación de cookies.")
        except NoSuchElementException: #Herramienta importada de Selenium.
            print(f"No se encontro la pagina para el ID {imdb_id}")
        except Exception as e:
            print(f"Error durante la búsqueda del ID IMDB {imdb_id}: {str(e)}")
         
         
        try:
            puntuacion_imdb = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-3a4309f8-0.bjXIAP.sc-b7c53eda-1.iIQkEw > div > div:nth-child(1) > a > span > div > div.sc-bde20123-0.dLwiNw > div.sc-bde20123-2.cdQqzc > span.sc-bde20123-1.cMEQkK").text
        except Exception as e:
            print(f"No se encontró la puntuación de {nombre_pelicula} en IMDb: {str(e)}")
            puntuacion_imdb = "N/A"
        
        try:
            director = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(1) > div").text
        except Exception as e:
            print(f"No se encontró el director de {nombre_pelicula} en IMDb: {str(e)}")
            director = "N/A"
        
        try:
            guionistas = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > div > section > div > div.sc-978e9339-1.ihWZgK.ipc-page-grid__item.ipc-page-grid__item--span-2 > section.ipc-page-section.ipc-page-section--base.sc-bfec09a1-0.jgUBLM.title-cast.title-cast--movie.celwidget > ul > li:nth-child(2) > div").text
        except Exception as e:
            print(f"No se encontró guionista de {nombre_pelicula} en IMDb: {str(e)}")
            guionistas = "N/A"
        
        try:
            try: 
                argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.iDcoFI > div.sc-92625f35-6.gHKhNg > div.sc-92625f35-10.dwKwMe > section > p > span.sc-7193fc79-2.kpMXpM").text
            except NoSuchElementException:
                try:
                    argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-10.hERErh > section > p > span.sc-7193fc79-0.ftEVcu").text
                except NoSuchElementException:
                    try:
                        argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.eqrmzR > div.sc-92625f35-6.cApiRN > div.sc-92625f35-10.hERErh > section > p > span.sc-7193fc79-2.kpMXpM").text
                    except NoSuchElementException:
                        try:
                            argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.kiNxNJ > div.sc-92625f35-6.gFpQdY > div.sc-92625f35-10.cDqtIi > section > p > span.sc-7193fc79-2.kpMXpM").text
                        except NoSuchElementException:
                            try:
                                argumento = driver.find_element("css selector", "#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-4.brwmyD > div > div.sc-92625f35-10.hERErh > section > p").text
                            except NoSuchElementException:
                                argumento = "N/A"
        except Exception as e:
          print(f"No se encontro argumento de {nombre_pelicula} en imdb {str(e)}")
          argumento = "N/A"       

        try:
            try:
                duracion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(3)").text
            except NoSuchElementException: # Esto lo ponemos porque hay paginas de imdb que tienen la direccion en diferentes paths.
                try:
                    duracion = driver.find_element("css selector","#__next > main > div > section.ipc-page-background.ipc-page-background--base.sc-304f99f6-0.fSJiHR > section > div:nth-child(5) > section > section > div.sc-92625f35-3.frxYSZ > div.sc-b7c53eda-0.dUpRPQ > ul > li:nth-child(2)").text
                except NoSuchElementException:
                 duracion = "N/A"
        except Exception as e:
                print(f"No se encontro duracion de {nombre_pelicula} en imdb {str(e)}")
                duracion = "N/A"
                
        # ROTTEN TOMATOES buscamos por nombre porque no acepta IDS a diferencia de IMD
        try:  # modificamos el nombre de las pelis para que se adapten a los parametros de búsqueda de 'Rotten Tomatoes'  Ya que está no funciona con búsqueda por ID'S.
            nombre_pelicula_modificado = nombre_pelicula.replace("'","").replace("ñ","n").replace(":","").replace(",","").replace("/","").replace("!","").replace("?","").replace(" ","_").replace("&","and").replace('á', 'a').replace('é', 'e').replace('í', 'i').replace('ó', 'o').replace('ú', 'u').replace('à', 'a').replace('è', 'e').replace('ì', 'i').replace('ò', 'o').replace('ù', 'u').lower()
            driver.get(f'https://www.rottentomatoes.com/m/{nombre_pelicula_modificado.replace(" ","_").replace("&","and").lower()}')
            sleep(2)
            try:
                driver.find_element("css selector", "#onetrust-accept-btn-handler").click()
            except NoSuchElementException:
            # Si el botón de aceptación de cookies no está presente, simplemente continúa. Lo cual, en este caso,  pasa a partir de la segunda busqueda.
                print("No se requiere aceptación de cookies.")
        
            puntuacion_rotten = driver.find_element("css selector", "#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button > rt-text").text
            
        except Exception as e:
            print(f"No se encontro la película {nombre_pelicula} en Rotten Tomatoes: {str(e)}")
            puntuacion_rotten = "N/A"
            

        datos_peliculas['Nombre'].append(nombre_pelicula)
        datos_peliculas['Puntuacion_IMDB'].append(puntuacion_imdb)
        datos_peliculas['Puntuacion_Rotten_Tomatoes'].append(puntuacion_rotten)
        datos_peliculas['Director'].append(director)
        datos_peliculas['Guionistas'].append(guionistas)
        datos_peliculas['Argumento'].append(argumento)
        datos_peliculas['Duracion'].append(duracion)

    driver.close()
    return datos_peliculas

# Llamamos a  la función con nuestro archivo de pelis que sacamos de la API.
df_peliculas = pd.read_csv("peliculas_api.csv")
resultados = extraccion_imdb_rotten(df_peliculas)





No se encontro la película Historia Lizy en Rotten Tomatoes: Message: no such element: Unable to locate element: {"method":"css selector","selector":"#modules-wrap > div.media-scorecard.no-border > media-scorecard > rt-button > rt-text"}
  (Session info: chrome=124.0.6367.119); For documentation on this error, please visit: https://www.selenium.dev/documentation/webdriver/troubleshooting/errors#no-such-element-exception
Stacktrace:
0   chromedriver                        0x0000000102fe68cc chromedriver + 4368588
1   chromedriver                        0x0000000102fded60 chromedriver + 4336992
2   chromedriver                        0x0000000102c02c04 chromedriver + 289796
3   chromedriver                        0x0000000102c44e00 chromedriver + 560640
4   chromedriver                        0x0000000102c7d5ec chromedriver + 792044
5   chromedriver                        0x0000000102c39ab4 chromedriver + 514740
6   chromedriver                        0x0000000102c3a50c chromedriver + 51

SystemExit: 0

/Users/patriciagonzalez/anaconda3/lib/python3.11/site-packages/IPython/core/interactiveshell.py:3561: UserWarning: To exit: use 'exit', 'quit', or Ctrl-D.
  warn("To exit: use 'exit', 'quit', or Ctrl-D.", stacklevel=1)


Este método añade contenido a un .csv existente sin reemplazar su contenido.